## Day 11: Dumbo Octopus

You enter a large cavern full of rare bioluminescent dumbo octopuses! They seem to not like the Christmas lights on your submarine, so you turn them off for now.

There are 100 octopuses arranged neatly in a 10 by 10 grid. Each octopus slowly gains energy over time and flashes brightly for a moment when its energy is full. Although your lights are off, maybe you could navigate through the cave without disturbing the octopuses if you could predict when the flashes of light will happen.

Each octopus has an energy level - your submarine can remotely measure the energy level of each octopus (your puzzle input). For example:

```
5483143223
2745854711
5264556173
6141336146
6357385478
4167524645
2176841721
6882881134
4846848554
5283751526
```

The energy level of each octopus is a value between 0 and 9. Here, the top-left octopus has an energy level of 5, the bottom-right one has an energy level of 6, and so on.

You can model the energy levels and flashes of light in steps. During a single step, the following occurs:

- First, the energy level of each octopus increases by 1.
- Then, any octopus with an energy level greater than 9 flashes. This increases the energy level of all adjacent octopuses by 1, including octopuses that are diagonally adjacent. If this causes an octopus to have an energy level greater than 9, it also flashes. This process continues as long as new octopuses keep having their energy level increased beyond 9. (An octopus can only flash at most once per step.)
- Finally, any octopus that flashed during this step has its energy level set to 0, as it used all of its energy to flash.

Adjacent flashes can cause an octopus to flash on a step even if it begins that step with very little energy. Consider the middle octopus with 1 energy in this situation:

Before any steps:

```
11111
19991
19191
19991
11111
```

After step 1:

```
34543
40004
50005
40004
34543
```

After step 2:

```
45654
51115
61116
51115
45654
```


An octopus is highlighted when it flashed during the given step.

Here is how the larger example above progresses:

Before any steps:

```
5483143223
2745854711
5264556173
6141336146
6357385478
4167524645
2176841721
6882881134
4846848554
5283751526
```

After step 1:

```
6594254334
3856965822
6375667284
7252447257
7468496589
5278635756
3287952832
7993992245
5957959665
6394862637
```

After step 2:

```
8807476555
5089087054
8597889608
8485769600
8700908800
6600088989
6800005943
0000007456
9000000876
8700006848
```



After 100 steps, there have been a total of 1656 flashes.

Given the starting energy levels of the dumbo octopuses in your cavern, simulate 100 steps. How many total flashes are there after 100 steps?



In [96]:
import numpy as np


def read_input(filename):
    """
    Load octopus input into a numpy array

    """

    with open(filename, "r") as infile:
        octopus_data = [list(map(int, l.strip())) for l in infile]

    return np.matrix(octopus_data)


def simulate_cave(filename = None, num_steps = 100, quiet = False, rand_size = 10):
    """
    Simulate octopus cave dynamics

    """
    
    if filename is None:
        # Generate random initial state
        cave_data = np.random.randint(low=0, high=7, size=(rand_size, rand_size))
    else:            
        # Load cave data from file
        cave_data = read_input(filename)

    # Keep track of the state of the cave at each step
    cave_states = []
    cave_states.append(cave_data)

    # print(cave_data)

    flash_count = 0

    # Iterate over specified number of steps
    for i in range(1, num_steps + 1):

        flash_step = 0

        # print("\n\nStep " + str(i) + ":\n")

        # Get cave state at start of this step
        cave_state = cave_states[i - 1].copy()

        # First, the energy level of each octopus increases by 1.
        cave_state += 1

        # Track which octopuses have not flashed yet during this step
        not_flashed = np.full(cave_state.shape, True)

        # While there are octopuses
        # - with an energy level greater than 9
        # - that have yet to flash...
        while np.logical_and(cave_state > 9, not_flashed).any():

            # Iterate over candidates for flashing
            flash_index = np.where(np.logical_and(cave_state > 9, not_flashed))
            for x in zip(flash_index[0], flash_index[1]):

                # Flash this octopus
                flash_count += 1
                flash_step += 1  # For part 2

                # Adjacent indices
                ind_adj = [
                    (x[0] + i, x[1] + j)
                    for i in [-1, 0, 1]
                    for j in [-1, 0, 1]
                    if x[0] + i in range(0, cave_state.shape[0])
                    and x[1] + j in range(0, cave_state.shape[1])
                    and (i, j) != (0, 0)
                ]

                # Increment each of the 8 adjacent elements by 1
                for y in ind_adj:
                    cave_state[y] += 1

                # An octopus can only flash at most once per step
                not_flashed[x] = False

        # Finally, any octopus that flashed during this step has its energy
        # level set to 0, as it used all of its energy to flash.
        cave_state[np.where(np.logical_not(not_flashed))] = 0

        # print(cave_state)
        # Part 2: 
        if flash_step == 100 and not quiet:
            print("Synchronised - step " + str(i))
            break
            

        # Write cave state
        cave_states.append(cave_state)
    
    if not quiet:
        print("Flash count after " + str(i) + " steps: " + str(flash_count))

    return cave_states


simulate_cave(filename = "day11-example.txt", num_steps = 100);


Flash count after 100 steps: 1656


In [95]:
simulate_cave(filename = "day11-input.txt", num_steps = 100);

Flash count after 100 steps: 1700


## Part Two

It seems like the individual flashes aren't bright enough to navigate. However, you might have a better option: the flashes seem to be synchronizing!

In the example above, the first time all octopuses flash simultaneously is step 195:

After step 195:

```
0000000000
0000000000
0000000000
0000000000
0000000000
0000000000
0000000000
0000000000
0000000000
0000000000
```

If you can calculate the exact moments when the octopuses will all flash simultaneously, you should be able to navigate through the cavern. What is the first step during which all octopuses flash?

In [12]:
simulate_cave(filename = "day11-input.txt", num_steps = 500);

Synchronised - step 273
Flash count after 273 steps: 4387


In [99]:
# For bonus stars, export a .gif of the octopus energy levels

import seaborn as sns;
sns.set_theme()
import os
import imageio

def plot_energy(filename = None, num_steps = 100, rand_size = 10):
    """
    Export .gif of octopus energy levels
    
    """

    cave_data = simulate_cave(filename, num_steps, quiet = True, rand_size = rand_size)
    
    filenames = []
    for i in range(0, len(cave_data)):
        ax = sns.heatmap(cave_data[i], 
                         vmin=0, 
                         vmax=9, 
                         yticklabels=False,
                         xticklabels=False,
                         cbar=False,
                         square=True)
        ax.text(0.1, 0.8,str(i), fontsize=20, fontfamily='monospace', color='#ffffff')
        filename = f'{i}.png'
        filenames.append(filename)
        plt = ax.get_figure()
        plt.savefig(filename, bbox_inches='tight', pad_inches=0)
        plt.clf()
    
    with imageio.get_writer('day11_random_100.gif', mode='I') as writer:
        for filename in filenames:
            image = imageio.imread(filename)
            writer.append_data(image)
        
    # Remove files
    for filename in set(filenames):
        os.remove(filename)
    
    return 0

plot_energy(filename = None, num_steps = 100, rand_size = 100);
    

<Figure size 432x288 with 0 Axes>

# gifs for bonus stars

Example data

![SegmentLocal](day11_example.gif "segment")

My input data

![SegmentLocal](day11_input.gif "segment")

Random 50x50 input data

![SegmentLocal](day11_random_50.gif "segment")

Random 100x100 input data

![SegmentLocal](day11_random_100.gif "segment")



